<a href="https://colab.research.google.com/github/Yanina-Kutovaya/GNN/blob/main/notebooks/Mock_NeighborLoader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mock-тест load_graph_from_postgres для NeighborLoader

Этот тестовый класс охватывает следующие аспекты работы функции load_graph_from_postgres:

- Проверка корректной загрузки структуры графа (узлы и рёбра).
- Реакция на отсутствие рёбер в данных.
- Обработка ошибок подключения или выполнения запросов к БД.
- Совместимость с инструментом NeighborLoader для последующего обучения графовых моделей.

Основные преимущества тестов:

- Исключают зависимость от реальной базы данных за счёт использования моков.
- Гарантируют устойчивость функции к различным сценариям, включая крайние случаи.
- Проверяют, что загруженный граф может быть использован в составе NeighborLoader — критично для задач обучения на основе соседних узлов в графах (GNN).

Небходимо выбрать среду выполнения с GPU: Среда выполнения → Сменить среду выполнения → Графический процессор T4

## 1. Установка зависимостей

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
install = True
if install:
  # 1. Установка совместимых версий PyTorch и PyG
  !pip install -q torch==2.3.0+cu121 torchvision==0.18.0+cu121 torchaudio==2.3.0+cu121 --extra-index-url https://download.pytorch.org/whl/cu121

  # 2. Установка зависимостей PyG для CUDA 12.1+ (совместимо с 12.5)
  !pip install -q pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html --no-cache-dir

  # 3. Установка PyTorch Geometric
  !pip install -q torch-geometric==2.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.0/781.0 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 83.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 84.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 107.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

__Проверка окружения:__

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")          # Должно быть 2.3.0+cu121
print(f"CUDA: {torch.version.cuda}")            # Должно быть 12.1+
print(f"Available: {torch.cuda.is_available()}")# Должно быть True


PyTorch: 2.3.0+cu121
CUDA: 12.1
Available: True


## 2. Полный тест с пояснениями

__Функция ```load_graph_from_postgres```__

В функцию ```load_graph_from_postgres```

* Добавлена обработка исключений вокруг pd.read_sql - Убедились, что даже если SQL-запросы завершаются ошибкой, функция не падает
* Убедились, что conn.close() вызывается в finally - Предотвращаем утечки соединений
* Возвращается Data(...) с пустыми тензорами - Обеспечивается совместимость с моделью, даже если данные не загружены

__Что проверяет каждый тест?__
- ```test_load_graph_handles_sql_query_errors``` - Корректный sql-запрос
- ```test_load_graph_returns_correct_data``` - Загрузка данных из БД с корректными узлами и рёбрами
- ```test_load_graph_returns_empty_edges``` - Обработка отсутствия рёбер в БД
- ```test_load_graph_handles_db_errors``` - Обработка ошибок подключения или SQL-запросов
- ```test_compatibility_with_neighbor_loader ``` - Проверка совместимости результата работы функции загрузки данных с NeighborLoader

In [ ]:
import logging
import unittest
from unittest.mock import patch, MagicMock
import pandas as pd
import torch
from torch_geometric.data import Data
import psycopg2
import warnings
from typing import Dict, Any

# Настройка логирования
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# === Тестируемая функция ===
def load_graph_from_postgres(db_config: Dict[str, Any]) -> Data:
    """
    Загружает графовые данные из PostgreSQL для использования в GNN-моделях через NeighborLoader.

    Args:
        db_config (Dict[str, Any]): Конфигурация для подключения к БД в формате:
            {
                'dbname': str,
                'user': str,
                'password': str,
                'host': str,
                'port': int
            }

    Returns:
        Data: Объект PyG с полями:
            - x: Признаки узлов (degree, total_received)
            - edge_index: Список рёбер в формате COO
            - edge_attr: Атрибуты рёбер (total_sent)
            - y: Метки классов узлов

    Raises:
        ValueError: При отсутствии обязательных столбцов в результатах SQL-запросов
        psycopg2.OperationalError: При проблемах подключения к БД
    """
    try:
        conn = psycopg2.connect(**db_config)
        logger.info("Установлено соединение с PostgreSQL")
        try:
            # Загрузка узлов
            query_nodes = "SELECT alias, label, degree, total_received FROM node_attributes"
            logger.debug(f"Выполняется SQL-запрос: {query_nodes}")
            nodes_df = pd.read_sql(query_nodes, conn)

            # Проверка структуры данных
            required_node_cols = ['alias', 'label', 'degree', 'total_received']
            if not all(col in nodes_df.columns for col in required_node_cols):
                raise ValueError("Отсутствуют обязательные колонки в таблице node_attributes")

            # Загрузка рёбер
            query_edges = "SELECT a, b, total_sent FROM edge_attributes"
            logger.debug(f"Выполняется SQL-запрос: {query_edges}")
            edges_df = pd.read_sql(query_edges, conn)

            # Проверка структуры данных
            required_edge_cols = ['a', 'b', 'total_sent']
            if not all(col in edges_df.columns for col in required_edge_cols):
                raise ValueError("Отсутствуют обязательные колонки в таблице edge_attributes")
        finally:
            if 'conn' in locals():
                conn.close()
                logger.info("Соединение с PostgreSQL закрыто")

        # Создание маппинга alias -> индекс
        alias_to_idx = {alias: idx for idx, alias in enumerate(nodes_df['alias'])}
        logger.debug(f"Создано {len(alias_to_idx)} маппингов alias->индекс")

        # Формирование рёбер
        edges = []
        edge_attrs = []

        for _, row in edges_df.iterrows():
            a_idx = alias_to_idx.get(row['a'])
            b_idx = alias_to_idx.get(row['b'])

            if a_idx is not None and b_idx is not None:
                edges.append([a_idx, b_idx])
                edge_attrs.append(row['total_sent'])
            else:
                logger.warning(f"Пропущено ребро с недействительными алиасами: {row['a']}->{row['b']}")

        # Убедимся, что edge_index и edge_attr — contiguous
        edge_index = torch.tensor(edges, dtype=torch.long).T.contiguous() if edges else torch.zeros((2, 0), dtype=torch.long)
        edge_attr = torch.tensor(edge_attrs, dtype=torch.float).view(-1, 1).contiguous() if edge_attrs else torch.zeros((0, 1), dtype=torch.float)

        # Формирование признаков узлов и меток
        x = torch.tensor(nodes_df[['degree', 'total_received']].fillna(0).values, dtype=torch.float)
        y = torch.tensor(nodes_df['label'].fillna(0).values, dtype=torch.long)

        logger.info(f"Загружен граф с {x.shape[0]} узлами и {edge_index.shape[1]} рёбрами")

        return Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)

    except psycopg2.OperationalError as e:
        logger.error(f"Ошибка подключения к БД: {str(e)}")
        # Возвращаем пустой граф при критических ошибках
        return Data(
            x=torch.tensor([], dtype=torch.float),
            edge_index=torch.tensor([], dtype=torch.long).view(2, -1),
            edge_attr=torch.tensor([], dtype=torch.float).view(-1, 1),
            y=torch.tensor([], dtype=torch.long)
        )

    except Exception as e:
        logger.exception(f"Неожиданная ошибка при загрузке графа: {str(e)}")
        # В случае ошибки возвращаем пустой граф
        return Data(
            x=torch.tensor([], dtype=torch.float),
            edge_index=torch.tensor([], dtype=torch.long).view(2, -1),
            edge_attr=torch.tensor([], dtype=torch.float).view(-1, 1),
            y=torch.tensor([], dtype=torch.long)
        )

# === Тестовый класс ===
class TestLoadGraphFromPostgres(unittest.TestCase):
    def setUp(self):
        # Конфигурация БД
        self.db_config = {
            'dbname': 'test_db',
            'user': 'test_user',
            'password': 'test_pass',
            'host': 'localhost',
            'port': 5432
        }
        # Фиктивные данные для узлов
        self.nodes_data = pd.DataFrame({
            'alias': [1, 2],
            'label': [0, 1],
            'degree': [2, 1],
            'total_received': [100.0, 200.0]
        })
        # Фиктивные данные для рёбер
        self.edges_data = pd.DataFrame({
            'a': [1, 2],
            'b': [2, 1],
            'total_sent': [50.0, 30.0]
        })

    # ✅ Тест: корректный sql-запрос
    @patch('logging.Logger.error')
    @patch('logging.Logger.exception')
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_load_graph_handles_sql_query_errors(
        self, mock_read_sql, mock_connect, mock_exception, mock_error
    ):
        mock_conn = MagicMock()
        mock_connect.return_value = mock_conn
        mock_read_sql.side_effect = psycopg2.ProgrammingError("Invalid SQL query")

        data = load_graph_from_postgres(self.db_config)

        self.assertIsInstance(data, Data)
        self.assertTrue(data.x.numel() == 0)
        self.assertTrue(data.edge_index.numel() == 0)
        self.assertTrue(data.edge_attr.numel() == 0)
        self.assertTrue(data.y.numel() == 0)

        # Проверяем, что logger.exception был вызван с правильным сообщением
        mock_exception.assert_called_once_with("Неожиданная ошибка при загрузке графа: Invalid SQL query")
        # Проверяем, что logger.error НЕ был вызван
        mock_error.assert_not_called()

    # ✅ Тест: корректная загрузка данных
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_load_graph_returns_correct_data(self, mock_read_sql, mock_connect):
        # Настройка мока для подключения к БД
        mock_conn = MagicMock()
        mock_cursor = MagicMock()
        mock_cursor.fetchone.return_value = [2]  # 2 узла
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        # Настройка мока для read_sql
        def side_effect(query, *args, **kwargs):
            if 'SELECT alias' in query:
                return self.nodes_data
            elif 'SELECT a, b' in query:
                return self.edges_data
            return pd.DataFrame()

        mock_read_sql.side_effect = side_effect

        # Вызов тестируемой функции
        data = load_graph_from_postgres(self.db_config)

        # Проверка структуры Data
        self.assertIsInstance(data, Data)
        self.assertEqual(data.x.shape, (2, 2))
        self.assertEqual(data.edge_index.shape, (2, 2))
        self.assertEqual(data.edge_attr.shape, (2, 1))
        self.assertEqual(data.y.shape, (2,))

        # Проверка содержимого
        expected_x = torch.tensor([[2, 100], [1, 200]], dtype=torch.float)
        expected_edge_index = torch.tensor([[0, 1], [1, 0]], dtype=torch.long)
        expected_edge_attr = torch.tensor([[50.0], [30.0]], dtype=torch.float)
        expected_y = torch.tensor([0, 1], dtype=torch.long)

        self.assertTrue(torch.equal(data.x, expected_x))
        self.assertTrue(torch.equal(data.edge_index, expected_edge_index))
        self.assertTrue(torch.equal(data.edge_attr, expected_edge_attr))
        self.assertTrue(torch.equal(data.y, expected_y))

    # ✅ Тест: обработка отсутствия рёбер
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_load_graph_returns_empty_edges(self, mock_read_sql, mock_connect):
        # Настройка мока
        mock_conn = MagicMock()
        mock_cursor = MagicMock()
        mock_cursor.fetchone.return_value = [2]
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        def side_effect(query, *args, **kwargs):
            if 'SELECT alias' in query:
                return self.nodes_data
            elif 'SELECT a, b' in query:
                return pd.DataFrame(columns=['a', 'b', 'total_sent'])
            return pd.DataFrame()

        mock_read_sql.side_effect = side_effect

        data = load_graph_from_postgres(self.db_config)

        # Проверка структуры
        self.assertIsInstance(data, Data)
        self.assertEqual(data.x.shape, (2, 2))
        self.assertEqual(data.edge_index.shape, (2, 0))
        self.assertEqual(data.edge_attr.shape, (0, 1))
        self.assertEqual(data.y.shape, (2,))

        # Проверка содержимого
        expected_x = torch.tensor([[2, 100], [1, 200]], dtype=torch.float)
        expected_edge_index = torch.zeros((2, 0), dtype=torch.long)
        expected_edge_attr = torch.zeros((0, 1), dtype=torch.float)
        expected_y = torch.tensor([0, 1], dtype=torch.long)

        self.assertTrue(torch.equal(data.x, expected_x))
        self.assertTrue(torch.equal(data.edge_index, expected_edge_index))
        self.assertTrue(torch.equal(data.edge_attr, expected_edge_attr))
        self.assertTrue(torch.equal(data.y, expected_y))

    # ✅ Тест: обработка ошибок БД
    @patch('logging.Logger.error')
    @patch('psycopg2.connect')
    def test_load_graph_handles_db_errors(self, mock_connect, mock_logger):
        mock_connect.side_effect = psycopg2.OperationalError("Connection failed")
        data = load_graph_from_postgres(self.db_config)

        # Проверяем, что возвращается пустой граф
        self.assertIsInstance(data, Data)
        self.assertTrue(data.x.numel() == 0)
        self.assertTrue(data.edge_index.numel() == 0)
        self.assertTrue(data.edge_attr.numel() == 0)
        self.assertTrue(data.y.numel() == 0)

        # Проверяем, что logger.error был вызван
        mock_logger.assert_called_once_with("Ошибка подключения к БД: Connection failed")


    # ✅ Тест: совместимость с NeighborLoader
    @patch('psycopg2.connect')
    @patch('pandas.read_sql')
    def test_compatibility_with_neighbor_loader(self, mock_read_sql, mock_connect):
        # Настройка мока для подключения к БД
        mock_conn = MagicMock()
        mock_cursor = MagicMock()
        mock_cursor.fetchone.return_value = [2]  # 2 узла
        mock_conn.cursor.return_value = mock_cursor
        mock_connect.return_value = mock_conn

        # Настройка мока для read_sql
        def side_effect(query, *args, **kwargs):
            if 'SELECT alias' in query:
                return self.nodes_data
            elif 'SELECT a, b' in query:
                return self.edges_data
            return pd.DataFrame()

        mock_read_sql.side_effect = side_effect

        # Вызов тестируемой функции
        data = load_graph_from_postgres(self.db_config)

        try:
            with warnings.catch_warnings():
                warnings.filterwarnings("ignore", message="An issue occurred while importing 'torch-sparse'.")
                warnings.filterwarnings("ignore", message="Using 'NeighborSampler' without a 'pyg-lib' installation is deprecated")

                from torch_geometric.loader import NeighborLoader
        except ImportError:
            self.skipTest("NeighborLoader not available (requires torch_geometric)")

        try:
            loader = NeighborLoader(
                data,
                num_neighbors=[-1],
                batch_size=2,
                input_nodes=torch.arange(data.num_nodes),
                shuffle=False  # Чтобы уменьшить неоднозначность
            )

            for batch in loader:
                self.assertIsInstance(batch, Data)

                # Проверка наличия ключевых атрибутов
                self.assertTrue(hasattr(batch, 'x'))
                self.assertTrue(hasattr(batch, 'edge_index'))
                self.assertTrue(hasattr(batch, 'edge_attr'))
                self.assertTrue(hasattr(batch, 'y'))

                # Проверка размеров
                self.assertEqual(batch.x.shape, (2, 2))
                self.assertEqual(batch.edge_index.shape[0], 2)  # 2 строки: [from, to]
                self.assertEqual(batch.edge_attr.shape, (2, 1))
                self.assertEqual(batch.y.shape, (2,))

                # Проверка значений признаков
                expected_x = torch.tensor([[2, 100], [1, 200]], dtype=torch.float)
                expected_edge_attr = torch.tensor([[50.0], [30.0]], dtype=torch.float32)
                expected_y = torch.tensor([0, 1], dtype=torch.long)

                self.assertTrue(torch.allclose(batch.x, expected_x, atol=1e-4))
                # Проверка значений edge_attr в любом порядке
                self.assertTrue(
                    torch.allclose(
                        batch.edge_attr.sort(dim=0).values,
                        expected_edge_attr.sort(dim=0).values,
                        atol=1e-3
                    )
                )
                self.assertTrue(torch.equal(batch.y, expected_y))

                # Проверка, что все индексы рёбер валидны
                self.assertLess(batch.edge_index.max(), batch.num_nodes)

                break
        except ImportError as e:
            error_msg = str(e)
            if "requires either 'pyg-lib' or 'torch-sparse'" in error_msg:
                self.skipTest("Missing optional packages: pyg-lib or torch-sparse")
            else:
                self.fail(f"Unexpected ImportError: {e}")
        except Exception as e:
            self.fail(f"NeighborLoader raised an unexpected exception: {e}")

## 3. Запуск тестов

In [ ]:
# === Запуск тестов ===
if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

.....
----------------------------------------------------------------------
Ran 5 tests in 0.062s

OK
